In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Fine-tuning T5 1.1 XL for abstractive summarization


This notebook demonstrates how to fine tune the T5 1.1 XL model for the abstractive  summarization task using the [XSum dataset](https://www.tensorflow.org/datasets/catalog/xsum).

## Imports and initialization

In [ ]:
# reloads modules automatically before executing any code/script
%load_ext autoreload
%autoreload 2

### Import libraries

Please refer to the [environment setup](../README.md) section in the README 
file to setup the development environment and install the required libraries 
before importing them.

In [ ]:
import os
import time
from datetime import datetime
import pandas as pd

import utils

# import vertex ai sdk for python
from google.cloud import aiplatform as vertex_ai

### Configure environment settings


- **`PROJECT_ID`:** Configure the Google Cloud Project ID
- **`REGION`:** Configure the [region](https://cloud.google.com/vertex-ai/docs/general/locations) 
  to be used for Vertex AI operations throughout the rest of this notebook
- **`BUCKET`:** Google Cloud Storage bucket name to be used by vertex AI for 
  any operations such as to stage the code, save any  artifacts generated etc.
- **`TENSORBOARD_NAME`:** Configure the managed TensorBoard instance name 
  created during the environment setup.

In [ ]:
# Project definitions
PROJECT_ID = '<YOUR PROJECT ID>' # Change to your project id.
REGION = '<YOUR REGION>'  # Change to your region.

# Bucket definitions
BUCKET = '<YOUR BUCKET NAME>' # Change to your bucket.

# Tensorboard definitions
TENSORBOARD_NAME = '<YOUR TENSORBOARD NAME>' # Change to your Tensorboard instance name

Get Vertex AI TensorBoard ID based on name.

In [ ]:
TENSORBOARD_ID = ! gcloud ai tensorboards list --filter="displayName={TENSORBOARD_NAME}" --format="value(name)" --region={REGION} 2>/dev/null 
TENSORBOARD_ID = TENSORBOARD_ID[0]

print(f"TENSORBOARD_ID = {TENSORBOARD_ID}")

### Configure custom container image

In this example, you use the base T5X custom training container.

In [ ]:
IMAGE_NAME = 't5x-base' 
IMAGE_URI = f'gcr.io/{PROJECT_ID}/{IMAGE_NAME}'

Validate image exists in the Container Registry

In [ ]:
! gcloud container images describe $IMAGE_URI

### Configure experiment settings


In [ ]:
EXPERIMENT_NAME = '<YOUR EXPERIMENT>' # Change to your experiment name

EXPERIMENT_WORKSPACE = f'gs://{BUCKET}/experiments/{EXPERIMENT_NAME}'
EXPERIMENT_RUNS = f'{EXPERIMENT_WORKSPACE}/runs'

### Initialize Vertex AI SDK for Python


In [ ]:
vertex_ai.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=EXPERIMENT_WORKSPACE,
    experiment=EXPERIMENT_NAME
)

### Configure dataset location


In [ ]:
TFDS_DATA_DIR = f'gs://{BUCKET}/datasets'

## Run fine-tuning job
### Define the job's gin file

This job is configured using the following Gin file.

In [ ]:
JOB_GIN_FILE = '../configs/finetune_t511_xl_xsum.gin'

!cat {JOB_GIN_FILE}

Following the hyper parameter settings reported in a number of papers, including [PEGASUS: Pre-training with Extracted Gap-sentences for
Abstractive Summarization](https://arxiv.org/pdf/1912.08777.pdf), the input length is set to 512 and the output length to 64. The batch size is set to 256.

The job is configured to run evaluations on the `validation` split of XSum every 500 steps. Since running evaluations on the full `validation` split would be compute intensive and would significantly extend the elapsed time, they are limited to 500 examples.

By default, `t5x/t5x/configs/runs/finetune.gin` does not put any constraints on the length of input and target features during evaluations. Their dimensions are computed by looking for the longest sequences in the data split used for evaluation. In the preprocessed  `validation` split of the XSum dataset, the longest sequence is 7730 tokens so the feature dimension of input batches would be set to 7730. The 90th percentile is 1061. To mitigate potential out of memory errors the maximum length of inputs for evaluation is constrained to 1024.

This configuration has been tested on a v2-128 TPU slice using an 8-way model parallelism, 16-way data parallelism.  





In [ ]:
GIN_FILES = [JOB_GIN_FILE]  
GIN_OVERWRITES = [
        'USE_CACHED_TASKS=False'
    ]

### Configure Vertex AI CustomJob

In [ ]:
RUN_NAME = f'<YOUR RUN NAME>' # Change to your run name for the custom job
RUN_ID = f'{EXPERIMENT_NAME}-{RUN_NAME}-{datetime.now().strftime("%Y%m%d%H%M")}'
RUN_DIR = f'{EXPERIMENT_RUNS}/{RUN_ID}'
RUN_MODE = 'train'

Log local variables defined for any troubleshooting

In [ ]:
for key in [
    "PROJECT_ID", "REGION", "BUCKET", "TENSORBOARD_NAME", "TENSORBOARD_ID", 
    "IMAGE_NAME", "IMAGE_URI", 
    "EXPERIMENT_NAME", "EXPERIMENT_WORKSPACE", "EXPERIMENT_RUNS", 
    "TFDS_DATA_DIR", "GIN_FILES", "GIN_OVERWRITES", 
    "RUN_NAME", "RUN_ID", "RUN_DIR", "RUN_MODE"
]:
    print(f"{key}={eval(key)}")

Configure a Cloud TPU slice for the job. Double check if your [region](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators) supports the specified TPU topology.

In [ ]:
MACHINE_TYPE = 'cloud-tpu'
ACCELERATOR_TYPE = 'TPU_V2'
ACCELERATOR_COUNT = 128

Create the custom job spec

In [ ]:
job = utils.create_t5x_custom_job(
    display_name=RUN_ID,
    machine_type=MACHINE_TYPE,
    accelerator_type=ACCELERATOR_TYPE,
    accelerator_count=ACCELERATOR_COUNT,
    image_uri=IMAGE_URI,
    run_mode=RUN_MODE,
    gin_files=GIN_FILES,
    model_dir=RUN_DIR,
    tfds_data_dir=TFDS_DATA_DIR,
    gin_overwrites=GIN_OVERWRITES
)

job.job_spec

### Submit the custom job to Vertex AI and track the experiment


In [ ]:
utils.submit_and_track_t5x_vertex_job(
    custom_job=job,
    job_display_name=RUN_ID,
    run_name=RUN_ID,
    experiment_name=EXPERIMENT_NAME,
    execution_name=RUN_ID,
    tfds_data_dir=TFDS_DATA_DIR,
    model_dir=RUN_DIR,
    vertex_ai=vertex_ai,
    run_mode=RUN_MODE
)

### Monitor the job with Vertex AI TensorBoard

**Execute the following command from the terminal window to sync logs to Vertex 
AI TensorBoard**

In [ ]:
cmd = f"""
tb-gcp-uploader --tensorboard_resource_name {TENSORBOARD_ID} \
--logdir {EXPERIMENT_RUNS} \
--experiment_name {EXPERIMENT_NAME}
"""

print(cmd)

To access the TensorBoard instance for the experiment, click the below URL

In [ ]:
TENSORBOARD_URL = f"https://{REGION}.tensorboard.googleusercontent.com/experiment/{TENSORBOARD_ID.replace('/', '+')}+experiments+{EXPERIMENT_NAME}/"
print(f"TensorBoard URL for the experiment is located at {TENSORBOARD_URL}")

Alternatively, you can access the Vertex AI TensorBoard experiment from the [console](https://console.cloud.google.com/vertex-ai/experiments/).